In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from math import sqrt
from datetime import datetime

In [2]:
cnx = sqlite3.connect('data/database.sqlite')
player_data = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
player_data.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [3]:
categorical_attributes = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']

numercial_attributes = ['potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

target_attribute = ['overall_rating']

In [4]:
player_data.shape

(183978, 42)

In [5]:
player_data.dropna(thresh=5, inplace=True)

In [6]:
player_data.shape

(183142, 42)

In [7]:
def generate_file_name(filename):
    now = datetime.now()
    current = now.strftime("%d_%m_%Y-%H_%M_%S")
    
    return now.strftime("%d_%m_%Y-%H_%M_%S") + '_' + filename 

In [8]:
from Clean_and_Merge import Clean_and_Merge

custom_attribute_object = Clean_and_Merge()

merged_player_data = custom_attribute_object.fit_transform(player_data)
merged_player_data.head()

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.600000,48.600000,43.600000,70.600000,60.600000,43.600000,50.600000,44.600000,38.600000,63.600000,...,66.000000,67.800000,5.600000,10.600000,9.600000,7.600000,7.600000,right,medium,medium
1,74.484848,70.787879,49.454545,52.939394,62.272727,29.151515,61.090909,61.878788,62.121212,63.242424,...,68.787879,71.515152,12.181818,8.666667,14.242424,10.363636,12.909091,left,medium,medium
2,74.192308,68.115385,57.923077,58.692308,65.115385,54.269231,69.038462,60.192308,55.615385,60.461538,...,21.115385,21.346154,14.038462,11.807692,17.730769,10.115385,13.500000,right,medium,medium
3,70.782609,57.217391,26.260870,69.260870,64.695652,47.782609,55.565217,37.782609,40.391304,60.826087,...,70.652174,68.043478,14.173913,11.173913,22.869565,11.173913,10.173913,right,medium,medium
4,74.680000,45.080000,38.840000,73.040000,64.760000,32.080000,50.600000,45.480000,26.360000,56.840000,...,76.040000,74.600000,8.280000,8.320000,24.920000,12.840000,11.920000,right,medium,medium


In [9]:
from Clean_and_Merge_Target import Clean_and_Merge_Target

merged_target_object = Clean_and_Merge_Target()
merged_target_data = merged_target_object.fit_transform(player_data)
merged_target_data.head()

,overall_rating
0,63.600000
1,66.969697
2,67.000000
3,69.086957
4,73.240000


In [10]:
merged_player_data

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.600000,48.600000,43.600000,70.600000,60.600000,43.600000,50.600000,44.600000,38.600000,63.600000,...,66.000000,67.800000,5.600000,10.600000,9.600000,7.600000,7.600000,right,medium,medium
1,74.484848,70.787879,49.454545,52.939394,62.272727,29.151515,61.090909,61.878788,62.121212,63.242424,...,68.787879,71.515152,12.181818,8.666667,14.242424,10.363636,12.909091,left,medium,medium
2,74.192308,68.115385,57.923077,58.692308,65.115385,54.269231,69.038462,60.192308,55.615385,60.461538,...,21.115385,21.346154,14.038462,11.807692,17.730769,10.115385,13.500000,right,medium,medium
3,70.782609,57.217391,26.260870,69.260870,64.695652,47.782609,55.565217,37.782609,40.391304,60.826087,...,70.652174,68.043478,14.173913,11.173913,22.869565,11.173913,10.173913,right,medium,medium
4,74.680000,45.080000,38.840000,73.040000,64.760000,32.080000,50.600000,45.480000,26.360000,56.840000,...,76.040000,74.600000,8.280000,8.320000,24.920000,12.840000,11.920000,right,medium,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,75.461538,42.000000,27.000000,75.153846,70.000000,30.000000,56.384615,49.000000,35.692308,54.538462,...,76.307692,72.692308,12.846154,12.769231,25.461538,11.384615,13.615385,right,low,medium
11056,71.625000,67.250000,46.750000,60.312500,63.125000,54.562500,61.187500,63.812500,61.750000,62.312500,...,65.250000,63.875000,9.000000,8.000000,5.000000,5.000000,8.000000,left,medium,medium
11057,72.857143,63.142857,44.571429,59.857143,66.857143,57.000000,66.857143,60.000000,52.571429,58.285714,...,64.285714,59.000000,7.142857,18.714286,42.428571,16.714286,16.714286,left,NaN,NaN
11058,78.125000,46.750000,43.000000,79.000000,58.875000,59.000000,36.000000,29.000000,41.375000,50.625000,...,73.250000,65.000000,14.000000,18.500000,41.625000,19.000000,20.000000,right,NaN,NaN


In [11]:
merged_player_data.shape

(11060, 37)

In [12]:
merged_target_data.shape

(11060, 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(merged_player_data, 
                                                    merged_target_data, 
                                                    test_size=0.20, 
                                                    random_state=101)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [15]:
num_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')), 
    ('num_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')), 
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_transformer = ColumnTransformer([
    ('num_transformer', num_pipeline, numercial_attributes),
    ('cat_transformer', cat_pipeline, categorical_attributes)
])

target_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean', add_indicator=True)), 
    ('num_scaler', StandardScaler())
])

In [16]:
full_transformed_data = full_transformer.fit_transform(X_train)
full_transformed_data

array([[ 0.58197118,  0.65919831,  1.21870086, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.75999394,  0.5594177 ,  1.61461829, ...,  0.        ,
         1.        ,  0.        ],
       [-0.58987965, -0.87256068, -1.07515648, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.61114926, -0.49402403, -0.94457582, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.77446541, -0.15817229, -0.54870336, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.91323318,  0.97212564,  0.74132583, ...,  0.        ,
         1.        ,  0.        ]])

In [17]:
full_transformed_target = target_pipeline.fit_transform(pd.DataFrame(y_train))
full_transformed_target

array([[ 0.81114056],
       [ 1.00651404],
       [-0.11224976],
       ...,
       [ 1.23316027],
       [ 0.31232155],
       [ 0.88329677]])

In [18]:
import joblib
import os

In [19]:
dir_name = 'pipelines'
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)
    
    
joblib.dump(custom_attribute_object, os.path.join(dir_name, generate_file_name('custom_attribute_object.pkl')))
joblib.dump(merged_target_object, os.path.join(dir_name, generate_file_name('merged_target_object.pkl')))    
joblib.dump(full_transformer, os.path.join(dir_name, generate_file_name('full_transformer.pkl')))
joblib.dump(target_pipeline, os.path.join(dir_name, generate_file_name('target_pipeline.pkl')))

['pipelines\\07_01_2022-22_02_52_target_pipeline.pkl']

In [20]:
full_transformed_data.shape

(8848, 43)

In [21]:
full_transformed_target.shape

(8848, 1)

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
%%time

lin_reg = LinearRegression()

cross_val_score(estimator=lin_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)



Wall time: 1.69 s


array([0.86573413, 0.87632903, 0.84636909, 0.86510393, 0.86442294,
       0.87188923, 0.86211915, 0.87657208, 0.86725601, 0.8658548 ])

In [24]:
%%time

tree_reg = DecisionTreeRegressor()

cross_val_score(estimator=tree_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

Wall time: 875 ms


array([0.88658672, 0.87280335, 0.85467317, 0.86108218, 0.86386048,
       0.87584259, 0.87511112, 0.86334545, 0.87586125, 0.85400728])

In [25]:
%%time

from sklearn.svm import SVR

svm_reg = SVR()

cross_val_score(estimator=svm_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

Wall time: 5.94 s


array([0.97685937, 0.97802457, 0.97273733, 0.97484917, 0.97245267,
       0.97649227, 0.97955977, 0.97637403, 0.97869964, 0.97274002])

In [26]:
%%time

from xgboost import XGBRegressor

xgb_reg = XGBRegressor(gpu_id=0, predictor='gpu_predictor')

cross_val_score(estimator=xgb_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

Wall time: 7.66 s


array([0.96058869, 0.961081  , 0.94811529, 0.95209434, 0.95416618,
       0.95389192, 0.96216504, 0.95794689, 0.9632156 , 0.95881319])

In [27]:
test_data = full_transformer.transform(X_test)
test_data

array([[ 1.0083062 ,  0.72221711,  1.49367349, ...,  0.        ,
         1.        ,  0.        ],
       [-1.2213958 ,  0.13374578,  0.28608535, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.91008926,  1.19957847,  0.04166657, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.96304828, -0.44522506,  0.64641407, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.5439789 ,  0.71530132, -1.33539389, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.05674455, -2.33776321, -1.88755753, ...,  0.        ,
         0.        ,  1.        ]])

In [28]:
best_model_details = {}

In [29]:
lin_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = lin_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[lin_reg] = error

Error :  0.35789459982440935


In [30]:
tree_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = tree_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[tree_reg] = error

Error :  0.3627541387158782


In [31]:
svm_reg.fit(X=full_transformed_data, y=full_transformed_target.ravel())

y_pred = svm_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[svm_reg] = error

Error :  0.14702290899009154


In [32]:
full_transformed_target.ravel()

array([ 0.81114056,  1.00651404, -0.11224976, ...,  1.23316027,
        0.31232155,  0.88329677])

In [33]:
xgb_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = xgb_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[xgb_reg] = error

Error :  0.1998373471295842


In [34]:
best_model_details

{LinearRegression(): 0.35789459982440935,
 DecisionTreeRegressor(): 0.3627541387158782,
 SVR(): 0.14702290899009154,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, predictor='gpu_predictor', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None): 0.1998373471295842}

In [35]:
min_value = list(best_model_details.values())[0]
my_model = ''

In [36]:
for key, value in best_model_details.items():
    if value < min_value:
        min_value = value
        my_model = key
        
print('Best performing model : {} \nError : {}'.format(my_model, min_value))

Best performing model : SVR() 
Error : 0.14702290899009154


In [37]:
dir_name = 'models'
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)
    
joblib.dump(my_model, os.path.join(dir_name, generate_file_name('best_model.pkl')))

['models\\07_01_2022-22_03_13_best_model.pkl']